In [1]:
import xlsxwriter
import datetime
import pandas as pd

In [2]:
today = datetime.date.today().strftime('%Y%m%d')

In [3]:
window = 486

In [4]:
fname = u'D:/report/工行私行/风险数据更新_%s.xlsx'%(today)

In [5]:
workbook = xlsxwriter.Workbook(fname)
worksheet = workbook.add_worksheet()

# 插入数值

### 设置格式

In [6]:
col_format = workbook.add_format({'bold': True, 'align': 'center', 'font_name': 'Microsoft yahei', 'font_color': '#FFFFFF', 'bg_color': '#993333'})
cell_format = workbook.add_format({'font_name': 'Microsoft yahei'})
percent_format = workbook.add_format({'font_name': 'Microsoft yahei', 'num_format': '0.00%'})
number_format = workbook.add_format({'font_name': 'Microsoft yahei', 'num_format': '0.00'})
float_format = workbook.add_format({'font_name': 'Microsoft yahei', 'num_format': '0.0000'})
highlight_format = workbook.add_format({'font_name': 'Microsoft yahei', 'num_format': '0.00', 'bg_color': '#FF3333'})
inter_format = workbook.add_format({'font_name': 'Microsoft yahei', 'num_format': '#,##0'})

In [7]:
worksheet.write('A1', u'指标', col_format)
worksheet.write('B1', u'数值', col_format)
worksheet.write('C1', u'标准化', col_format)

0

### 持仓盈亏与盈亏占比

In [8]:
df = pd.read_excel('D:/workspace/zjsxzy_in_js/website/everyday-update/data/881001.WI.xlsx')

In [9]:
df['current return z'] = df['current return'].rolling(window).apply(lambda x: (x[-1] - x.mean()) / x.std())
df['profit percentage z'] = df['profit percentage'].rolling(window).apply(lambda x: (x[-1] - x.mean()) / x.std())

In [10]:
worksheet.write('A2', u'持有成本盈亏', cell_format)
worksheet.write('B2', df['current return'][-1], percent_format)
if df['current return z'][-1] > 2 or df['current return z'][-1] < -2:
    worksheet.write('C2', df['current return z'][-1], highlight_format)
else:
    worksheet.write('C2', df['current return z'][-1], number_format)

In [11]:
worksheet.write('A3', u'持仓盈亏占比', cell_format)
worksheet.write('B3', df['profit percentage'][-1], percent_format)
if df['profit percentage z'][-1] > 2 or df['profit percentage z'][-1] < -2:
    worksheet.write('C2', df['profit percentage z'][-1], highlight_format)
else:
    worksheet.write('C3', df['profit percentage z'][-1], number_format)

### 回购加权成交期限

In [12]:
repo = pd.read_excel('D:/Data/risk/weighted_repo.xlsx')

In [13]:
repo = repo.dropna()

In [14]:
repo['z'] = repo['rolling mean'].rolling(window).apply(lambda x: (x[-1] - x.mean()) / x.std())

In [15]:
worksheet.write('A4', u'回购加权成交期限', cell_format)
worksheet.write('B4', repo['weighted'][-1], number_format)
if repo['z'][-1] > 2 or repo['z'][-1] < -2:
    worksheet.write('C4', repo['z'][-1], highlight_format)
else:
    worksheet.write('C4', repo['z'][-1], number_format)

### 股票流动性风险

In [16]:
st_li = pd.read_excel('D:/workspace/zjsxzy_in_js/website/everyday-update/data/amihud_liquidity.xlsx')

In [17]:
st_li['z'] = st_li['wdqa_corwin and schultz'].rolling(window).apply(lambda x: (x[-1] - x.mean()) / x.std())

In [18]:
worksheet.write('A5', u'股票流动性风险', cell_format)
worksheet.write('B5', st_li['wdqa_corwin and schultz'][-1], float_format)
if st_li['z'][-1] > 2 or st_li['z'][-1] < -2:
    worksheet.write('C5', st_li['z'][-1], highlight_format)
else:
    worksheet.write('C5', st_li['z'][-1], number_format)

### 人民币与全球资产系统性风险

In [19]:
risk = pd.read_excel('D:/Data/risk/systematic.xlsx')

In [20]:
risk['China z'] = risk['China'].rolling(window).apply(lambda x: (x[-1] - x.mean()) / x.std())
risk['Global z'] = risk['Global'].rolling(window).apply(lambda x: (x[-1] - x.mean()) / x.std())

In [21]:
worksheet.write('A6', u'人民币资产系统性风险', cell_format)
worksheet.write('B6', risk['China'][-1], percent_format)
if risk['China z'][-1] > 2 or risk['China z'][-1] < -2:
    worksheet.write('C6', risk['China z'][-1], highlight_format)
else:
    worksheet.write('C6', risk['China z'][-1], number_format)

In [22]:
worksheet.write('A7', u'全球资产系统性风险', cell_format)
worksheet.write('B7', risk['Global'][-1], percent_format)
if risk['Global z'][-1] > 2 or risk['Global z'][-1] < -2:
    worksheet.write('C7', risk['Global z'][-1], highlight_format)
else:
    worksheet.write('C7', risk['Global z'][-1], number_format)

### 股债相对收益率

In [23]:
eyby = pd.read_excel('D:/workspace/zjsxzy_in_js/website/everyday-update/data/EYBY.xlsx')

In [24]:
eyby = eyby.dropna()

In [25]:
eyby['spread'] = ((100. / eyby['pe_ttm']) - eyby['ytm_b']).rolling(window=5).mean()
eyby['spread'] = eyby['spread'] / 100

In [26]:
eyby['z'] = eyby['spread'].rolling(window=window).apply(lambda x: (x[-1] - x.mean()) / x.std())

In [27]:
worksheet.write('A8', u'股债相对收益率', cell_format)
worksheet.write('B8', eyby['spread'][-1], percent_format)
if eyby['z'][-1] > 2 or eyby['z'][-1] < -2:
    print('highlight')
    worksheet.write('C8', eyby['z'][-1], highlight_format)
else:
    worksheet.write('C8', eyby['z'][-1], number_format)

highlight


### 市场方向一致度

In [28]:
cons = pd.read_excel('D:/workspace/zjsxzy_in_js/website/everyday-update/data/consistency.xlsx', index_col=0)

In [29]:
cons = cons[cons.index >= '2005-01-01']

In [30]:
cons['z'] = cons['con60'].rolling(window=window).apply(lambda x: (x[-1] - x.mean()) / x.std())

In [31]:
worksheet.write('A9', u'市场方向一致度', cell_format)
worksheet.write('B9', cons['con60'][-1], percent_format)
if cons['z'][-1] > 2 or cons['z'][-1] < -2:
    worksheet.write('C9', cons['z'][-1], highlight_format)
else:
    worksheet.write('C9', cons['z'][-1], number_format)

### 债券流动性

In [32]:
bo_li = pd.read_excel('D:/Data/risk/bond_liquidity.xlsx', index_col=0)

In [33]:
bo_li['z'] = bo_li['ILLIQUIDITY'].rolling(window=window).apply(lambda x: (x[-1] - x.mean()) / x.std())

In [34]:
worksheet.write('A10', u'债券流动性风险', cell_format)
worksheet.write('B10', bo_li['ILLIQUIDITY'][-1], number_format)
if bo_li['z'][-1] > 2 or bo_li['z'][-1] < -2:
    worksheet.write('C10', bo_li['z'][-1], highlight_format)
else:
    worksheet.write('C10', bo_li['z'][-1], number_format)

### 散户点击量

In [35]:
click = pd.read_excel(u'D:/Data/risk/点击量数据.xlsx')

In [36]:
click.tail()

,Date,Page Clicks,# of Users
1487,2018-10-18,2801008.0,834626.0
1488,2018-10-19,2819385.0,788511.0
1489,2018-10-22,3316930.0,942521.0
1490,2018-10-23,2933565.0,825684.0
1491,2018-10-24,2718604.0,762896.0


In [37]:
click = click.set_index('Date')

In [38]:
click['click'] = click['Page Clicks'].rolling(20).mean()

In [39]:
click['z'] = click['click'].rolling(window).apply(lambda x: (x[-1] - x.mean()) / x.std())

In [40]:
worksheet.write('A11', u'散户点击量', cell_format)
worksheet.write('B11', click['Page Clicks'][-1], inter_format)
if click['z'][-1] > 2 or click['z'][-1] < -2:
    worksheet.write('C11', click['z'][-1], highlight_format)
else:
    worksheet.write('C11', click['z'][-1], number_format)

# 美欧经济意外指数之差

In [41]:
us_eu = pd.read_excel('D:/Data/macro/us/econ_accident.xlsx')

In [42]:
us_eu['diff'] = us_eu['us econ ac'] - us_eu['eu econ ac']

In [43]:
us_eu['z'] = us_eu['diff'].rolling(window).apply(lambda x: (x[-1] - x.mean()) / x.std())

In [44]:
worksheet.write('A12', u'美欧经济意外指数之差', cell_format)
worksheet.write('B12', us_eu['diff'][-1], number_format)
if us_eu['z'][-1] > 2 or us_eu['z'][-1] < -2:
    worksheet.write('C12', us_eu['z'][-1], highlight_format)
else:
    worksheet.write('C12', us_eu['z'][-1], number_format)

# 铜油比

In [45]:
cop = pd.read_excel(u'D:/Data/industry/metal/LME铜三个月期货价格.xlsx')
oil = pd.read_excel(u'D:/Data/industry/metal/Brent油价.xlsx')

In [46]:
cop_oil_ratio = (cop[cop.columns[0]] / oil[oil.columns[0]]).dropna()

In [47]:
cop_oil_z = cop_oil_ratio.rolling(window).apply(lambda x: (x[-1] - x.mean()) / x.std())

In [48]:
worksheet.write('A13', u'铜油比', cell_format)
worksheet.write('B13', cop_oil_ratio[-1], number_format)
if cop_oil_z[-1] > 2 or cop_oil_z[-1] < -2:
    worksheet.write('C13', cop_oil_z[-1], highlight_format)
else:
    worksheet.write('C13', cop_oil_z[-1], number_format)

### 设置第一列列宽

In [49]:
worksheet.set_column(0, 0, 20)
worksheet.set_column(1, 1, 15)

0

# 插入图片

In [50]:
worksheet.insert_image('A16', 'plot.png',
                        {'x_scale': 0.85, 'y_scale': 0.85})

In [51]:
workbook.close()